In [1]:
import numpy as np
from numpy import pi
from qiskit import QuantumCircuit, assemble, Aer
from qiskit.visualization import plot_bloch_multivector, plot_histogram
sim = Aer.get_backend('aer_simulator')

In [2]:
def c_amod15(a, power):
    """Controlled multiplication by a mod 15"""
    if a not in [2,4,7,8,11,13]:
        raise ValueError("'a' must be 2,4,7,8,11 or 13")
    U = QuantumCircuit(4)        
    for iteration in range(power):
        if a in [2,13]:
            U.swap(0,1)
            U.swap(1,2)
            U.swap(2,3)
        if a in [7,8]:
            U.swap(2,3)
            U.swap(1,2)
            U.swap(0,1)
        if a in [4, 11]:
            U.swap(1,3)
            U.swap(0,2)
        if a in [7,11,13]:
            for q in range(4):
                U.x(q)
    U = U.to_gate()
    U.name = "%i^%i mod 15" % (a, power)
    c_U = U.control()
    return c_U

In [11]:
n_count = 9
a = 8

In [9]:
def qft_dagger(n):
    """n-qubit QFTdagger the first n qubits in circ"""
    qc = QuantumCircuit(n)
    # Don't forget the Swaps!
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    qc.name = "QFT†"
    return qc

In [12]:
# Create QuantumCircuit with n_count counting qubits
# plus 4 qubits for U to act on
qc = QuantumCircuit(n_count + 4, n_count)

# Initialize counting qubits
# in state |+>
for q in range(n_count):
    qc.h(q)
    
# And auxiliary register in state |1>
qc.x(3+n_count)

# Do controlled-U operations
for q in range(n_count):
    qc.append(c_amod15(a, 2**q), 
             [q] + [i+n_count for i in range(4)])

# Do inverse-QFT
qc.append(qft_dagger(n_count), range(n_count))

# Measure circuit
qc.measure(range(n_count), range(n_count))
qc.draw(fold=-1)  # -1 means 'do not fold' 

┌───┐                                                                                                                                              ┌───────┐┌─┐                        
 q_0: ┤ H ├───────■──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤0      ├┤M├────────────────────────
      ├───┤       │                                                                                                                                      │       │└╥┘┌─┐                     
 q_1: ┤ H ├───────┼──────────────■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤1      ├─╫─┤M├─────────────────────
      ├───┤       │              │                                                                                                                       │       │ ║ └╥┘┌─┐                  
 q_2: ┤ H ├───────┼──────────────┼──────────────■────────────────────────────────────────────────────────────────────────────────────────────────────────┤2      ├─╫──╫─┤M├──────────────────
      ├───┤       │              │              │                                                                                                        │       │ ║  ║ └╥┘┌─┐               
 q_3: ┤ H ├───────┼──────────────┼──────────────┼──────────────■─────────────────────────────────────────────────────────────────────────────────────────┤3      ├─╫──╫──╫─┤M├───────────────
      ├───┤       │              │              │              │                                                                                         │       │ ║  ║  ║ └╥┘┌─┐            
 q_4: ┤ H ├───────┼──────────────┼──────────────┼──────────────┼──────────────■──────────────────────────────────────────────────────────────────────────┤4 QFT† ├─╫──╫──╫──╫─┤M├────────────
      ├───┤       │              │              │              │              │                                                                          │       │ ║  ║  ║  ║ └╥┘┌─┐         
 q_5: ┤ H ├───────┼──────────────┼──────────────┼──────────────┼──────────────┼───────────────■──────────────────────────────────────────────────────────┤5      ├─╫──╫──╫──╫──╫─┤M├─────────
      ├───┤       │              │              │              │              │               │                                                          │       │ ║  ║  ║  ║  ║ └╥┘┌─┐      
 q_6: ┤ H ├───────┼──────────────┼──────────────┼──────────────┼──────────────┼───────────────┼───────────────■──────────────────────────────────────────┤6      ├─╫──╫──╫──╫──╫──╫─┤M├──────
      ├───┤       │              │              │              │              │               │               │                                          │       │ ║  ║  ║  ║  ║  ║ └╥┘┌─┐   
 q_7: ┤ H ├───────┼──────────────┼──────────────┼──────────────┼──────────────┼───────────────┼───────────────┼────────────────■─────────────────────────┤7      ├─╫──╫──╫──╫──╫──╫──╫─┤M├───
      ├───┤       │              │              │              │              │               │               │                │                         │       │ ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐
 q_8: ┤ H ├───────┼──────────────┼──────────────┼──────────────┼──────────────┼───────────────┼───────────────┼────────────────┼────────────────■────────┤8      ├─╫──╫──╫──╫──╫──╫──╫──╫─┤M├
      └───┘┌──────┴──────┐┌──────┴──────┐┌──────┴──────┐┌──────┴──────┐┌──────┴───────┐┌──────┴───────┐┌──────┴───────┐┌───────┴───────┐┌───────┴───────┐└───────┘ ║  ║  ║  ║  ║  ║  ║  ║ └╥┘
 q_9: ─────┤0            ├┤0            ├┤0            ├┤0            ├┤0             ├┤0             ├┤0             ├┤0              ├┤0              ├──────────╫──╫──╫──╫──╫──╫──╫──╫──╫─
           │             ││             ││             ││             ││              ││              ││              ││               ││               │          ║  ║  ║  ║  ║  ║  ║  ║  ║ 
q_10: ─────┤1   